In [4]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
user_user_links_part1 = pd.read_csv("/content/drive/MyDrive/final/users/user_user_links_part1.csv")
user_user_links_part2 = pd.read_csv("/content/drive/MyDrive/final/users/user_user_links_part2.csv")
user_user_links_part3 = pd.read_csv("/content/drive/MyDrive/final/users/user_user_links_part3.csv")
user_user_links_part4 = pd.read_csv("/content/drive/MyDrive/final/users/user_user_links_part4.csv")
user_user_links_part5 = pd.read_csv("/content/drive/MyDrive/final/users/user_user_links_part5.csv")

In [ ]:
norm_factor_aa = max(user_user_links_part1["aa"].max(), user_user_links_part2["aa"].max(), user_user_links_part3["aa"].max(), user_user_links_part4["aa"].max(), user_user_links_part5["aa"].max())
norm_factor_aa


148.27088200059762

In [ ]:
user_user_links_part1["aa"] = user_user_links_part1["aa"]/norm_factor_aa
user_user_links_part2["aa"] = user_user_links_part2["aa"]/norm_factor_aa
user_user_links_part3["aa"] = user_user_links_part3["aa"]/norm_factor_aa
user_user_links_part4["aa"] = user_user_links_part4["aa"]/norm_factor_aa
user_user_links_part5["aa"] = user_user_links_part5["aa"]/norm_factor_aa

In [ ]:
len(user_user_links_part5)

3647556

In [ ]:
user_user_links = user_user_links_part5
hyperparameters = {"cmrs":0.2,"genre":0.2,"cn":0.,"aa":0.1,"hpi":0.025,"jc":0.025,"ra":0.01,"si":0.01,"lhn":0.01,"scs":0.01,"hdi":0.01,"user_feature_sim":0.4}
net_sim = []
for i in range(len(user_user_links)):
  sim = 0
  for key in hyperparameters.keys():
    sim += hyperparameters[key]*user_user_links[key][i]
  net_sim.append(sim)
  if i%300000==0:
    print("Done with:", i)

user_user_links_part5["weighted_similarity"] = net_sim
user_user_links_part5.head()

Done with: 0
Done with: 300000
Done with: 600000
Done with: 900000
Done with: 1200000
Done with: 1500000
Done with: 1800000
Done with: 2100000
Done with: 2400000
Done with: 2700000
Done with: 3000000
Done with: 3300000
Done with: 3600000


,node1,node2,user_feature_sim,cn,jc,genre,cmrs,aa,hpi,ra,si,lhn,scs,hdi,weighted_similarity
0,u1017,u4983,0.968515,22,0.047414,0.508285,0.925820,0.022708,0.200000,0.041798,0.045267,0.000532,0.108176,0.058511,0.685226
1,u1017,u3641,0.931434,71,0.105185,0.677775,0.943219,0.074847,0.191892,0.157701,0.095174,0.000510,0.190355,0.188830,0.718010
2,u1017,u5309,0.476393,2,0.005141,0.589307,0.970143,0.001862,0.133333,0.001484,0.005115,0.000355,0.026631,0.005319,0.506484
3,u1017,u808,0.925692,108,0.221311,0.972150,0.922968,0.143993,0.490909,1.100976,0.181208,0.001306,0.375507,0.287234,0.800968
4,u1017,u2259,0.974705,35,0.068093,0.532198,0.952819,0.034316,0.202312,0.040703,0.063752,0.000538,0.137231,0.093085,0.700430


In [ ]:
user_user_links_part5.to_csv("/content/drive/MyDrive/final/users/user_user_links_part5.csv", index=False)

In [ ]:
user_user_links_part1 = user_user_links_part1.drop(columns=["user_feature_sim","cn","jc","genre","cmrs","aa","hpi","ra","si","lhn","scs","hdi"])
user_user_links_part2 = user_user_links_part2.drop(columns=["user_feature_sim","cn","jc","genre","cmrs","aa","hpi","ra","si","lhn","scs","hdi"])
user_user_links_part3 = user_user_links_part3.drop(columns=["user_feature_sim","cn","jc","genre","cmrs","aa","hpi","ra","si","lhn","scs","hdi"])
user_user_links_part4 = user_user_links_part4.drop(columns=["user_feature_sim","cn","jc","genre","cmrs","aa","hpi","ra","si","lhn","scs","hdi"])
user_user_links_part5 = user_user_links_part5.drop(columns=["user_feature_sim","cn","jc","genre","cmrs","aa","hpi","ra","si","lhn","scs","hdi"])
uul = pd.concat([user_user_links_part1,user_user_links_part2,user_user_links_part3,user_user_links_part4,user_user_links_part5], ignore_index=True)

In [ ]:
print(len(uul))
uul.head(3)

18237780


,node1,node2,weighted_similarity
0,u780,u1665,0.752623
1,u780,u1950,0.776857
2,u780,u6004,0.732917


In [ ]:
uul.to_csv("/content/drive/MyDrive/final/users/user_net_similarity.csv", index=False)

In [ ]:
uul = pd.read_csv("/content/drive/MyDrive/final/users/user_net_similarity.csv")

In [ ]:
uul.head(3)

,node1,node2,weighted_similarity
0,u780,u1665,0.752623
1,u780,u1950,0.776857
2,u780,u6004,0.732917


In [ ]:
node1_set = set(uul["node1"])
print(len(node1_set))
node2_set = set(uul["node2"])
print(len(node2_set))
node2_excl = node2_set - node1_set
print(len(node2_excl))

6039
6039
1


In [ ]:
node2_excl

{'u1497'}

In [ ]:
user_nodes = node1_set.union(node2_excl)
user_nodes = list(user_nodes)
print(len(user_nodes))

6040


In [ ]:
user_nodes[:2]

['u3451', 'u2099']

In [ ]:
from heapq import nlargest

NUM = 10
top_n_similar_users = []
count = 0
for i in user_nodes:
  # as node 1
  top_n = {}
  df1 = uul[uul["node1"]==i]
  df1 = df1.sort_values("weighted_similarity", axis=0, ascending=False).head(10)
  # print(df1)
  for k in df1.index:
    top_n[df1["node2"][k]] = df1["weighted_similarity"][k]
  
  # as node 2
  df2 = uul[uul["node2"]==i] 
  df2 = df2.sort_values("weighted_similarity", axis=0, ascending=False).head(10)
  for k in df2.index:
    top_n[df2["node1"][k]] = df2["weighted_similarity"][k]

  # print(top_n)
  top_n = nlargest(NUM, top_n, key = top_n.get)
  top_n_similar_users.append([i,top_n])
  count += 1
  if count%200 == 0:
    print("Done with: {} users".format(count))
  # break
# print(top_n_similar_users)
# df
top_n_users = pd.DataFrame(top_n_similar_users)

Done with: 200 users
Done with: 400 users
Done with: 600 users
Done with: 800 users
Done with: 1000 users
Done with: 1200 users
Done with: 1400 users
Done with: 1600 users
Done with: 1800 users
Done with: 2000 users
Done with: 2200 users
Done with: 2400 users
Done with: 2600 users
Done with: 2800 users
Done with: 3000 users
Done with: 3200 users
Done with: 3400 users
Done with: 3600 users
Done with: 3800 users
Done with: 4000 users
Done with: 4200 users
Done with: 4400 users
Done with: 4600 users
Done with: 4800 users
Done with: 5000 users
Done with: 5200 users
Done with: 5400 users
Done with: 5600 users
Done with: 5800 users
Done with: 6000 users


In [ ]:
top_n_users.to_csv("/content/drive/MyDrive/final/users/training_set_top_10_similar_users.csv", index=False)

In [ ]:
top_n_users.columns = ['user_id','top_10_similar_users']

In [5]:
top_n_users = pd.read_csv("/content/drive/MyDrive/final/users/training_set_top_10_similar_users.csv")

In [6]:
len(top_n_users)

6040

In [8]:
top_n_users.describe()

,user_id,top_10_similar_users
count,6040,6040
unique,6040,6040
top,u3042,"['u3099', 'u445', 'u516', 'u3761', 'u2088', 'u..."
freq,1,1
